# Multimodal

## Setup

### Imports

In [4]:
# Imports
import os
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm
from collections import OrderedDict
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import torchmetrics
from src.utils import *
from src.models import MLPSTILClassifier, Attention1DSTILClassifier, Attention2DSTILClassifier
from src.train import train_mm, evaluate, train_mm_stil_classifier

%load_ext autoreload
%autoreload 2

### Set seed & device

In [5]:
set_seed(42)      
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Extract text feats from reports

In [6]:
%%script false --no-raise-error
# load biobert model & tokenizer
tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-large-cased-v1.1-mnli')
lm = AutoModel.from_pretrained('dmis-lab/biobert-large-cased-v1.1-mnli')
data_dir = '/mnt/disks/ext/data/gdc/tcga/brca'
output_dir = 'data/report_feats'
extract_text_features(lm, tokenizer, data_dir)

## Task: Predict sTILs from WSIs & reports

### Set hparams

In [7]:
bsz = 16 # batch size for dataloaders
img_channels_in = 2048  # emb dim of wsi feats
text_channels_in = 1024 # emb dim of report feats
hidden_dim_mlp = 32 # hidden layer dim of mlp
num_classes = 10 # number of stil levels

### Load data

In [8]:
# Create the dataset
root_dir = './'
data_file = 'data/stils/data_stils.csv'
train_data = MMDataset(root_dir, data_file, 'train')
val_data = MMDataset(root_dir, data_file, 'val')
test_data = MMDataset(root_dir, data_file, 'test')

# Create the dataloaders
train_loader = DataLoader(train_data, batch_size=bsz, shuffle=True, num_workers=4, collate_fn=MMDataset.mm_collate_fn)
val_loader = DataLoader(val_data, batch_size=bsz, shuffle=False, num_workers=4, collate_fn=MMDataset.mm_collate_fn)
test_loader = DataLoader(test_data, batch_size=bsz, shuffle=False, num_workers=4, collate_fn=MMDataset.mm_collate_fn)

### Train & eval

#### MLP classifier

In [10]:
# %%script false --no-raise-error
# init model
mlp_model = MLPSTILClassifier(img_channels_in, text_channels_in, hidden_dim_mlp, num_classes).to(device)

# set training args
args = {'num_epochs': 20}

# train model
# model = train_mm(mlp_model, train_loader, val_loader, num_epochs, device)
mlp_model, trainer = train_mm_stil_classifier(mlp_model, train_loader, val_loader, args)

# evaluate the trained model on the test set
# test_loss, test_acc = evaluate(model, test_loader, device)
trainer.test(mlp_model, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/envs/multimodal/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/sta

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Metric val_loss improved. New best score: 2.293


...

Metric val_loss improved by 0.005 >= min_delta = 0.0. New best score: 1.990
`Trainer.fit` stopped: `max_epochs=20` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


training on device: cpu


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.37681159377098083    │
│      test_acc_epoch       │    0.4598016142845154     │
│         test_loss         │     2.087115526199341     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 2.087115526199341,
  'test_acc': 0.37681159377098083,
  'test_acc_epoch': 0.4598016142845154}]

#### MIL classifier w 1D attention

In [11]:
# Initialize the model
model = Attention1DSTILClassifier(img_channels_in, text_channels_in, num_classes)

# set training args
args = {'patience': 5, 'num_epochs': 20, 'ckpt_dir': 'model_ckpts'}

# Train the model
model, trainer = train_mm_stil_classifier(model, train_loader, val_loader, args)

# Evaluate the model
trainer.test(model, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type               | Params
--------------------------------------------------
0 | attention  | Sequential         | 262 K 
1 | classifier | Sequential         | 30.7 K
2 | loss       | CrossEntropyLoss   | 0     
3 | accuracy   | MulticlassAccuracy | 0     
--------------------------------------------------
293 K     Trainable params
0         Non-trainable params
293 K     Total params
1.173     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Metric val_loss improved. New best score: 1.962


...

Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 1.961
`Trainer.fit` stopped: `max_epochs=20` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


training on device: cpu


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.37681159377098083    │
│      test_acc_epoch       │    0.49875009059906006    │
│         test_loss         │    2.0843870639801025     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 2.0843870639801025,
  'test_acc': 0.37681159377098083,
  'test_acc_epoch': 0.49875009059906006}]

#### MIL classifier w 2D attention

In [12]:
# Initialize the model
model = Attention2DSTILClassifier(img_channels_in, text_channels_in, num_classes)

# set training args
args = {'patience': 5, 'num_epochs': 20, 'ckpt_dir': 'model_ckpts'}

# Train the model
model, trainer = train_mm_stil_classifier(model, train_loader, val_loader, args)

# Evaluate the model
trainer.test(model, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type                    | Params
-------------------------------------------------------
0 | attention  | EncoderDecoderAttention | 1.3 M 
1 | maps       | Sequential              | 18.4 K
2 | classifier | Sequential              | 30.7 K
3 | loss       | CrossEntropyLoss        | 0     
4 | accuracy   | MulticlassAccuracy      | 0     
-------------------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.287     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Metric val_loss improved. New best score: 1.961


...

Monitored metric val_loss did not improve in the last 5 records. Best score: 1.961. Signaling Trainer to stop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


training on device: cpu


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.37681159377098083    │
│      test_acc_epoch       │    0.4980216324329376     │
│         test_loss         │    2.0843751430511475     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 2.0843751430511475,
  'test_acc': 0.37681159377098083,
  'test_acc_epoch': 0.4980216324329376}]